# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
output_file= "../output_data/cities.csv"

cities_df= pd.read_csv(output_file)
cities_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
humidity= cities_df["Humidity"].astype(float)
maxhumidity= humidity.max()
locations= cities_df[["Lat", "Lng"]]

In [ ]:
figure = gmaps.figure()
heat= gmaps.heatmap_layer(locations, weights = humidity, dissipating= False, max_intensity=maxhumidity, point_radius=2)
figure.add_layer(heat)
figure

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_weather= cities_df[(cities_df['Max Temp'].between(70,77)) & (cities_df['Wind Speed']<5)].dropna()
ideal_weather.reset_index(drop=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df= []


for x in range(len(ideal_weather)):
    lat = ideal_weather.iloc[x]['Lat']
    leng= ideal_weather.iloc[x]['Lng']
    
    
    print(lat)
    print(leng)
    params= {"location": f"{lat},{leng}",
            'radius': 5000, 
            'types': "hotel",
            "key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request= requests.get(base_url, params= params)
    response= request.json()
    try:
        hotel_df.append(response['results'][0]['name'])
    except:
        hotel_df.append("")

In [ ]:
Hotel_Name_df=pd.DataFrame({'Hotel Name': hotel_df}) 
Hotel_Name_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers= gmaps.marker_layer(locations)
figure.add_layer(markers)

# Display figure
figure